In [7]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
import os
import yaml
import torch
import logging
from typing import List, Dict, Any

import pandas as pd
from datasets import load_dataset, Dataset
from transformers import pipeline
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer

import evaluate

class SummarizationEvaluator:
    def __init__(self, config: Dict[str, Any]):
        """Initialize summarization evaluator"""
        self.config = config
        self.logger = self._setup_logger()
        
        # Set up device
        self.device = 0 if torch.cuda.is_available() else -1
        
        # Load evaluation metrics
        self.rouge = evaluate.load('rouge')
        self.bleu = evaluate.load('bleu')
        self.meteor = evaluate.load('meteor')
        self.bertscore = evaluate.load('bertscore')
    
    def _setup_logger(self):
        """Set up logging configuration"""
        logging.basicConfig(
            level=logging.INFO, 
            format='%(asctime)s - %(levelname)s: %(message)s'
        )
        return logging.getLogger(__name__)
    
    def load_dataset(self, subset: bool = False) -> Dataset:
        """Load dataset from configuration"""
        self.logger.info("Loading dataset...")
        dataset = load_dataset(
            self.config['DATASET_PATH'], 
            split="validation"
        )
        
        if subset:
            dataset = Dataset.from_dict(dataset[:10])
        
        return dataset
    
    def load_summarization_pipeline(self, model_path: str) -> Any:
        """Load summarization model pipeline"""
        self.logger.info(f"Loading summarization pipeline: {model_path}")
        tokenizer_path = "tiiuae/Falcon3-1B-Instruct"
        
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.bfloat16,
        )
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    
        return pipeline(
            "text-generation", 
            model=model, 
            tokenizer=tokenizer,
            device=self.device,
            torch_dtype=torch.bfloat16,
        )
    
    def generate_summaries(
        self, 
        dataset: Dataset, 
        model_path: str, 
        text_column: str = "text",
        batch_size: int = 16
    ) -> Dataset:
        """Generate summaries in batches"""
        summarizer = self.load_summarization_pipeline(model_path)
        model_name = model_path.split('/')[-1]
        
        input_texts = dataset[text_column]
        summaries = []
        
        is_sft = 'SFT' in model_name or 'Instruct' in model_name
        
        for i in tqdm(range(0, len(input_texts), batch_size), desc=f'Summarizing {model_name}'):
            batch_inputs = input_texts[i:i+batch_size]
            
            # Apply the formatting
            if is_sft:
                formatted_inputs = [
                    f"### System: You are a summarization tool designed exclusively to generate concise summaries of Arabic input text.\n### Human: Summarize the following text in Arabic:\n{source_text}\n### Assistant:" 
                    for source_text in batch_inputs
                ]
            else:
                formatted_inputs = batch_inputs
            
            # Generate summaries
            batch_summaries = summarizer(
                formatted_inputs, 
                do_sample=False, 
                max_new_tokens=128
            )
            
            batch_summaries = [
                summary[0]['generated_text'] for summary in batch_summaries
            ]
            
            summaries.extend(batch_summaries)
        
        dataset = dataset.add_column(f"summary_{model_name}", summaries)
        return dataset

    def compute_metrics(
        self, 
        dataset: Dataset, 
        models: List[str], 
        tokenizer, 
        reference_column: str = "summary"
    ) -> pd.DataFrame:
        """Compute comprehensive metrics with preprocessing"""
        results = []

        for model_path in models:
            model_name = model_path.split('/')[-1]
            summary_column = f"summary_{model_name}"
            
            # Extract predictions and references
            predictions = dataset[summary_column]
            references = dataset[reference_column]

            # Preprocess predictions and references
            predictions = [
                (p if p.endswith(("!", "؟", "۔")) else p + "۔") for p in predictions
            ]
            references = [
                (r if r.endswith(("!", "؟", "۔")) else r + "۔") for r in references
            ]

            sent_tokenizer = RegexpTokenizer(u'[^!؟۔]*[!؟۔]')
            predictions = [
                "\n".join([s.strip() for s in sent_tokenizer.tokenize(p)]) for p in predictions
            ]
            references = [
                "\n".join([s.strip() for s in sent_tokenizer.tokenize(r)]) for r in references
            ]
            
            # Compute ROUGE
            rouge_results = self.rouge.compute(
                predictions=predictions, 
                references=references
            )
            
            # Compute BLEU
            bleu_results = self.bleu.compute(
                predictions=predictions, 
                references=references
            )
            
            # Compute METEOR
            meteor_results = self.meteor.compute(
                predictions=predictions, 
                references=references
            )
            
            # Compute BERTScore
            bertscore_results = self.bertscore.compute(
                predictions=predictions, 
                references=references, 
                lang='ar'
            )
            
            # Aggregate results
            model_metrics = {
                "model": model_name,
                "rouge1": rouge_results["rouge1"] * 100,
                "rouge2": rouge_results["rouge2"] * 100,
                "rougeL": rouge_results["rougeL"] * 100,
                "rougeLsum": rouge_results["rougeLsum"] * 100,
                "bleu": bleu_results["bleu"] * 100,
                "meteor": meteor_results["meteor"] * 100,
                "bertscore_precision": sum(bertscore_results['precision']) / len(bertscore_results['precision']) * 100,
                "bertscore_recall": sum(bertscore_results['recall']) / len(bertscore_results['recall']) * 100,
                "bertscore_f1": sum(bertscore_results['f1']) / len(bertscore_results['f1']) * 100
            }
            
            results.append(model_metrics)
        
        return pd.DataFrame(results)

    
    def compute_metrics(
        self, 
        dataset: Dataset, 
        models: List[str], 
        reference_column: str = "summary"
    ) -> pd.DataFrame:
        """Compute comprehensive metrics"""
        results = []
        
        for model_path in models:
            model_name = model_path.split('/')[-1]
            summary_column = f"summary_{model_name}"
            
            # Extract predictions and references
            predictions = dataset[summary_column]
            references = dataset[reference_column]
            
            # Compute ROUGE
            rouge_results = self.rouge.compute(
                predictions=predictions, 
                references=references
            )
            
            # Compute BLEU
            bleu_results = self.bleu.compute(
                predictions=predictions, 
                references=references
            )
            
            # # Compute METEOR
            # meteor_results = self.meteor.compute(
            #     predictions=predictions, 
            #     references=references
            # )
            
            # Compute BERTScore
            # bertscore_results = self.bertscore.compute(
            #     predictions=predictions, 
            #     references=references, 
            #     lang='ar'
            # )
            
            # Aggregate results
            model_metrics = {
                "model": model_name,
                "rouge1": rouge_results["rouge1"] * 100,
                "rouge2": rouge_results["rouge2"] * 100,
                "rougeL": rouge_results["rougeL"] * 100,
                "rougeLsum": rouge_results["rougeLsum"] * 100,
                "bleu": bleu_results["bleu"] * 100,
                # "meteor": meteor_results["meteor"] * 100,
                # "bertscore_precision": sum(bertscore_results['precision']) / len(bertscore_results['precision']) * 100,
                # "bertscore_recall": sum(bertscore_results['recall']) / len(bertscore_results['recall']) * 100,
                # "bertscore_f1": sum(bertscore_results['f1']) / len(bertscore_results['f1']) * 100
            }
            
            results.append(model_metrics)
        
        return pd.DataFrame(results)
    
    def save_results(
        self, 
        dataset: Dataset, 
        metrics_df: pd.DataFrame, 
        output_path: str
    ):
        """Save results to disk"""
        os.makedirs(output_path, exist_ok=True)
        
        self.logger.info(f"Saving results to {output_path}...")
        
        dataset_df = dataset.to_pandas()
        dataset_df.to_csv(os.path.join(output_path, "summarized_dataset.csv"), index=False)
        metrics_df.to_csv(os.path.join(output_path, "metrics.csv"), index=False)
    
    def run_evaluation(self, subset: bool = False):
        """Run complete evaluation pipeline"""
        dataset = self.load_dataset(subset)
        
        for model_path in self.config['EVALUATE_MODELS']:
            dataset = self.generate_summaries(
                dataset, 
                model_path, 
                text_column=self.config['TEXT_COLUMN']
            )
            
            # Free GPU memory after each model evaluation as we do not need the model to stay in VRAM
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        metrics_df = self.compute_metrics(
            dataset, 
            self.config['EVALUATE_MODELS']
        )
        
        self.save_results(
            dataset, 
            metrics_df, 
            self.config.get('OUTPUT_PATH', './results')
        )
        
        return dataset, metrics_df

In [26]:
df = pd.read_csv("summarized_dataset.csv")

In [27]:
df

,text,summary,summary_model_name,tokenizer_name,dataset_source,sequence_length,summary_Qwen2.5-0.5B-Instruct-bs-2-lr-2e-05-ep-1-wp-100-gacc-32-gnm-1.0-FP16-SFT-mx-2048-v2
0,يوجد الكثير من الأساليب التي تتعدى ممارسة الجن...,في العلاقات السليمة، يمكنك بناء الثقة والتقارب...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/wikihow-arabic-summarization,969,في العلاقات السليمة، يمكن استخدام الأنشطة المخ...
1,لقد ظهر أن التمارين الفترية عالية الشدة (HIIT)...,تساعد التمارين الفترية عالية الشدة على فقدان ا...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/wikihow-arabic-summarization,1171,تشير النصوص إلى أهمية التمارين الرياضية الفتري...
2,عبد الحكيم حذاقة-الجزائر عاشت الجزائر مرحلة مف...,شهدت الجزائر خلال أكتوبر-ديسمبر 2019 ثورة سلمي...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/arabic-article-summarization,1887,شهدت الجزائر ثورة سلمية في عام 2019، لكنها
3,يحتاج صديقك في أغلب الأوقات لعناق كبير يطمئنه ...,تتضمن النصائح تقديم العناق والمس، الاستماع، تق...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/wikihow-arabic-summarization,2439,### System: You are a summarization tool desig...
4,إن كان الطبيب أو الأخصائي يعتقد بأن رقبتك مستق...,في حالة استقرار الرقبة، يمكن بدء العلاج الطبيع...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/wikihow-arabic-summarization,1587,في بداية الجلسة، تطلب الأخصائي تغيير رقبتك لتشمل
...,...,...,...,...,...,...,...
432,حدد ماهية المشكلة قبل القيام بإعادة تثبيت كامل...,للحل قبل إعادة تثبيت نظام Windows بشكل كامل:\n...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/wikihow-arabic-summarization,1552,في إعادة تثبيت Windows 7، قم بتحديد المشكلة وا...
433,الذهاب مبكرا جدا إلى المستشفى قد يؤدي إلى تدخل...,الذهاب مبكرا للمستشفى قد يؤدي لتدخلات غير ضرور...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/wikihow-arabic-summarization,1125,تشير النصوص إلى أهمية الذهاب مبكرا إلى المستشف...
434,قبل الصعود على مركب (كبيرا كان أم صغيرا) حاول ...,قبل الإبحار، تجنب الأطعمة الثقيلة والدهنية وال...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/wikihow-arabic-summarization,1465,قبل الصعود على مركبة البحرية، يجب الحرص على تج...
435,لكل شخص حق في الحصول على وتقبل المساعدة، كما أ...,حق الشخص في الحصول والرفض للمساعدة، مع إمكانية...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/wikihow-arabic-summarization,1281,لكل شخص حق في الحصول على وتقدير المساعدة، ويمك...


In [28]:
df[df["sequence_length"] <1024]

,text,summary,summary_model_name,tokenizer_name,dataset_source,sequence_length,summary_Qwen2.5-0.5B-Instruct-bs-2-lr-2e-05-ep-1-wp-100-gacc-32-gnm-1.0-FP16-SFT-mx-2048-v2
0,يوجد الكثير من الأساليب التي تتعدى ممارسة الجن...,في العلاقات السليمة، يمكنك بناء الثقة والتقارب...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/wikihow-arabic-summarization,969,في العلاقات السليمة، يمكن استخدام الأنشطة المخ...
5,عندما يفقد الجسم الحرارة أسرع مما يكتسبها يكون...,عند انخفاض درجة حرارة الجسم عن 35 درجة مئوية، ...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/wikihow-arabic-summarization,679,إن انخفاض حرارة الجسم يؤدي إلى مشاكل صحية خطير...
7,يمكنك الحصول على قصة الشعر التي تعجبك من خلال ...,يمكنك الحصول على قصة الشعر التي تعجبك من خلال ...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/wikihow-arabic-summarization,741,يمكن الوصول إلى قصة الشعر الخاصة بك من خلال تو...
10,\nدعا النائب بمجلس نواب الشعب وليد جلاّد رئيسة...,دعا النائب وليد جلاّد رئيسة هيئة الحقيقة و الك...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/Arabic-article-summarization-30-000,302,دعا النائب جلاًد رئيسة هيئة الحقيقة و الكرامة ...
14,البورسلين معرض للخدش بسهولة، لذا تجنب استخدام ...,تجنب استخدام لوفات التنظيف على البورسلين واستخ...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/wikihow-arabic-summarization,740,بروسبين معرض للخدش، استخدم إسفنجة نظيفة و
...,...,...,...,...,...,...,...
422,دعا الرئيس الفرنسي إيمانويل ماكرون الأوروبيين ...,دعا ماكرون الأوروبيين إلى تحرك فعال في مواجهة ...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/arabic-article-summarization,636,دعا الرئيس الفرنسي إيمانويل ماكرون الأوروبيين ...
425,\nتنقل الرادار اليوم الأربعاء 11 جانفي 2017 إل...,تنقل رادار الفحص الفني للسيارات في سوسa إلى مر...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/Arabic-article-summarization-30-000,249,تتواصل الرادار مع مركز الفحص الفني لنقل السيار...
428,هذا الفخ الشائع والزهيد متاح في معظم متاجر الع...,فخ غراء لصيد الثعابين يتضمن صندوق الغراء وطعام...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/wikihow-arabic-summarization,427,الفخ الشائع والمفيد هو صندوق يمكن وضعه تحت الغ...
430,\nتداول رواد مواقع التواصل الاجتماعي صوراً لشا...,شاركت أشخاص على الإنترنت في الترويج لشاب إيران...,Qwen/Qwen2.5-7B-Instruct-AWQ,Qwen/Qwen2.5-7B-Instruct-AWQ,Abdelkareem/Arabic-article-summarization-30-000,296,طلب رضا برستش لزيارة ميسي بسبب تطابق وجهه مع


In [29]:
df["summary_Qwen2.5-0.5B-Instruct-bs-2-lr-2e-05-ep-1-wp-100-gacc-32-gnm-1.0-FP16-SFT-mx-2048-v2"].iloc[2]


'شهدت الجزائر ثورة سلمية في عام 2019، لكنها'

In [30]:
"Assistant" in df["summary_Qwen2.5-0.5B-Instruct-bs-2-lr-2e-05-ep-1-wp-100-gacc-32-gnm-1.0-FP16-SFT-mx-2048-v2"].iloc[80]

False

In [ ]:
df["summary_Qwen2.5-0.5B-Instruct-bs-2-lr-2e-05-ep-1-wp-100-gacc-32-gnm-1.0-FP16-SFT-mx-2048-v2"].iloc[0]


In [2]:
with open('eval_config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [3]:
evaluator = SummarizationEvaluator(config)
dataset, metrics = evaluator.run_evaluation(subset=False)

print("Model Performance Metrics:")
print(metrics)

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2025-01-26 23:09:56,251 - INFO: Loading dataset...
2025-01-26 23:09:58,722 - INFO: Loading summarization pipeline: BounharAbdelaziz/Falcon3-1B-Base-bs-2-lr-2e-05-ep-3-wp-100-gacc-32-gnm-1.0-FP16-mx-2048-lora-r-128-a-32
Device set to use cuda:0
Summarizing Falcon3-1B-Base-bs-2-lr-2e-05-ep-3-wp-100-gacc-32-gnm-1.0-FP16-mx-2048-lora-r-128-a-32:   0%|          | 0/28 [00:00<?, ?it/s]This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, 

Model Performance Metrics:
                                               model     rouge1    rouge2  \
0  Falcon3-1B-Base-bs-2-lr-2e-05-ep-3-wp-100-gacc...  10.534322  4.505363   

      rougeL  rougeLsum      bleu  
0  10.468481  10.428355  1.578146  


In [4]:
dataset

Dataset({
    features: ['text', 'summary', 'summary_model_name', 'tokenizer_name', 'dataset_source', 'sequence_length', 'summary_Falcon3-1B-Base-bs-2-lr-2e-05-ep-3-wp-100-gacc-32-gnm-1.0-FP16-mx-2048-lora-r-128-a-32'],
    num_rows: 437
})

In [5]:
dataset[0]

{'text': 'يوجد الكثير من الأساليب التي تتعدى ممارسة الجنس لتكوني قريبة من الطرف الآخر في العلاقات السليمة. يمكنك بناء الثقة والتقارب في العلاقة من خلال التحدث والاستماع والمشاركة واحترام أفكاركما ومن خلال الخروج معا والاستمتاع بصحبة بعضكما البعض. إذا حاول شريكك إخبارك بأن الجنس هو الطريقة الوحيدة التي يمكنكما التقارب بها، فقد تحتاجين إلى التفكير فيما إذا كانت هذه العلاقة هي ما تريدينه حقا. من الممتع أن تجلسي مع شريكك وتقوما بنشاطات غير جنسية. يعد اللعب بألعاب لوحية وسيلة رائعة لإراحة شريكك وتخفيف توتره دون الحاجة لممارسة الجنس. اجلسا بجانب بعضكما البعض واستندا على بعض الوسائد وتناوبا على القراءة. يمكنكما حتى اختيار الكتب بالتناوب. يعتمد قرب جلوسكما على درجة راحتك.  إذا كنت قلقة بشأن تطور الأمور، فحاولي أن تبتعدي قليلا. يمكنك الجلوس مثلا على طرف مقابل له مع تلامس قدميكما فقط. يمكنكما قراءة بعض القصص الرومانسية لبعضكما من أجل تشتيت الانتباه بعيدا عن الجنس (وربما كي تضحكا على مدى رداءة القصص). اجلسا معا واستمعا إلى ألبوم. ابحثي عن كلمات الأغاني واجلسي بالقرب من شريكك وتمتعي بقرب جسدكما. و

In [6]:
metrics

,model,rouge1,rouge2,rougeL,rougeLsum,bleu
0,Falcon3-1B-Base-bs-2-lr-2e-05-ep-3-wp-100-gacc...,10.534322,4.505363,10.468481,10.428355,1.578146


In [52]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
import torch
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [53]:
def load_model_and_tokenizer(path):
    model = AutoModelForCausalLM.from_pretrained(
        path,
        torch_dtype=torch.bfloat16,
    ).to("cuda")

    model.eval()

    tokenizer = AutoTokenizer.from_pretrained(path)
    
    return model, tokenizer

In [62]:
model_path = "BounharAbdelaziz/Qwen2.5-0.5B-Instruct-bs-2-lr-1e-05-ep-1-wp-100-gacc-32-gnm-1.0-FP16-SFT-mx-2048-v2"
model, tokenizer = load_model_and_tokenizer(model_path)
model_name = model_path.split('/')[-1]
max_length = 2048
max_new_tokens = 2048

In [65]:
dataset = load_dataset("BounharAbdelaziz/Arabic-Synthetic-Summarization-Dataset-Filtered", split="test")

NameError: name 'load_dataset' is not defined

In [59]:
test_input = """يوجد الكثير من الأساليب التي تتعدى ممارسة الجنس لتكوني قريبة من الطرف الآخر في العلاقات السليمة. يمكنك بناء الثقة والتقارب في العلاقة من خلال التحدث والاستماع والمشاركة واحترام أفكاركما ومن خلال الخروج معا والاستمتاع بصحبة بعضكما البعض. إذا حاول شريكك إخبارك بأن الجنس هو الطريقة الوحيدة التي يمكنكما التقارب بها، فقد تحتاجين إلى التفكير فيما إذا كانت هذه العلاقة هي ما تريدينه حقا. من الممتع أن تجلسي مع شريكك وتقوما بنشاطات غير جنسية. يعد اللعب بألعاب لوحية وسيلة رائعة لإراحة شريكك وتخفيف توتره دون الحاجة لممارسة الجنس. اجلسا بجانب بعضكما البعض واستندا على بعض الوسائد وتناوبا على القراءة. يمكنكما حتى اختيار الكتب بالتناوب. يعتمد قرب جلوسكما على درجة راحتك. إذا كنت قلقة بشأن تطور الأمور، فحاولي أن تبتعدي قليلا. يمكنك الجلوس مثلا على طرف مقابل له مع تلامس قدميكما فقط. يمكنكما قراءة بعض القصص الرومانسية لبعضكما من أجل تشتيت الانتباه بعيدا عن الجنس (وربما كي تضحكا على مدى رداءة القصص). اجلسا معا واستمعا إلى ألبوم. ابحثي عن كلمات الأغاني واجلسي بالقرب من شريكك وتمتعي بقرب جسدكما. ولكن لا تفعلا أكثر من مشابكة أياديكما أو احتضان بعضكما. اكتبا استطلاعات رأي لبعضكما البعض أو اطبعا بعضا منها مسبقا من الإنترنت (ابحثا عن "أسئلة ممتعة لشريكك"). اجلسا على جانبين متقابلين وليجب كل منكما بالإجابة على استبيانه وحده، ثم تبادلا الاستبيانين بمجرد الانتهاء لقراءة الردود. حضرا بعض الوجبات الخفيفة والكثير من الوسائد لمشاهدة فيلم ما. إذا كنت قلقة من تحول الفيلم إلى جلسة حميمية، ضعي الوسائد بينك وبينه واستخدما بطانيتين منفصلتين. إذا لم تؤدي الوسائد والبطانيتين وظيفتهما، فلا بأس من سؤال شريكك إذا كان ينوي فعل شيء آخر يستدعي مقاومتك. حولي المكان الذي ستجلسان فيه إلى حصن. أضيفي بعض الأضواء والبطانيات الأنيقة ثم اجلسا في داخله وشاهدا فيلما أو اقرآ بعض القصص. قد يؤدي جلوسك بالقرب من شريكك إلى صعوبة في مقاومة تطور الأمر لأكثر من مجرد احتضان. إذا وجدت نفسك تفكرين في تجاوز حدودكما المحددة مسبقا، ابتعدي عن شريكك أو اقترحي الجلوس في مكان آخر. إذا كنت قلقة من الإساءة إلى شريكك بابتعادك عنه، فيمكنك التذرع لرغبتك في الانتقال على سبيل المثال بحجة مثل: "ظهري يؤلمني من الجلوس هنا، دعنا نجلس على أريكة أخرى" أو "أشعر ببعض الضيق، لنأخذ استراحة ونذهب للمشي!" سوف يساعدك الخروج إلى الأماكن العامة على كبح رغباتك الجنسية. إذا تطورت بينكما الأمور لوضع مثير وشعرت بالسوء حيال الأمر، لا بأس في التوقف وقول إنك لا تشعرين بالراحة في الاستمرار. يجب أن يحترم شريكك رغباتك على الفور. إذا حاول شريكك إقناعك بتجاوز الحدود التي وضعتها بشكل متكرر، فقد تحتاجين إلى التفكير في استمرار علاقتك معه. على الرغم من أنه من المقبول أن يسألك شريكك عن رأيك مرة أخرى بين الحين والآخر، إلا أنه لا ينبغي أن يشعرك بالسوء نحو قرارك."""

def predict_summary(test_input, model, tokenizer, max_length, max_new_tokens, model_name):

    # custom instruct prompt start
    prompt_template_test = f"Summarize this arabic text:\n{{text}}\n---\nSummary:\n"

    test_prompt = prompt_template_test.format(text=test_input)
    print(test_prompt)
    
    # Tokenize and generate
    inputs = tokenizer(test_prompt, return_tensors="pt", max_length=max_length).to("cuda")
    outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            # num_beams=4,
            # early_stopping=True,
            # no_repeat_ngram_size=3,
            # repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id,  # Crucial for stopping
            # pad_token_id=tokenizer.eos_token_id,  # Crucial for stopping
            # do_sample=True,
        )


    # Decode the output
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    cleaned_summary = summary.split("Summary:")[-1].strip()
    # print('-'*100)
    # print(f"{summary}")
    # print('-'*100)
    # print(f"{cleaned_summary}")
    
    return {f"summary_{model_name}": cleaned_summary}

In [63]:
_ = predict_summary(test_input, model, tokenizer, max_length, max_new_tokens, model_name)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Summarize this arabic text:
يوجد الكثير من الأساليب التي تتعدى ممارسة الجنس لتكوني قريبة من الطرف الآخر في العلاقات السليمة. يمكنك بناء الثقة والتقارب في العلاقة من خلال التحدث والاستماع والمشاركة واحترام أفكاركما ومن خلال الخروج معا والاستمتاع بصحبة بعضكما البعض. إذا حاول شريكك إخبارك بأن الجنس هو الطريقة الوحيدة التي يمكنكما التقارب بها، فقد تحتاجين إلى التفكير فيما إذا كانت هذه العلاقة هي ما تريدينه حقا. من الممتع أن تجلسي مع شريكك وتقوما بنشاطات غير جنسية. يعد اللعب بألعاب لوحية وسيلة رائعة لإراحة شريكك وتخفيف توتره دون الحاجة لممارسة الجنس. اجلسا بجانب بعضكما البعض واستندا على بعض الوسائد وتناوبا على القراءة. يمكنكما حتى اختيار الكتب بالتناوب. يعتمد قرب جلوسكما على درجة راحتك. إذا كنت قلقة بشأن تطور الأمور، فحاولي أن تبتعدي قليلا. يمكنك الجلوس مثلا على طرف مقابل له مع تلامس قدميكما فقط. يمكنكما قراءة بعض القصص الرومانسية لبعضكما من أجل تشتيت الانتباه بعيدا عن الجنس (وربما كي تضحكا على مدى رداءة القصص). اجلسا معا واستمعا إلى ألبوم. ابحثي عن كلمات الأغاني واجلسي بالقرب من شريكك وتمت

In [64]:
_

{'summary_Qwen2.5-0.5B-Instruct-bs-2-lr-1e-05-ep-1-wp-100-gacc-32-gnm-1.0-FP16-SFT-mx-2048-v2': 'يتم بناء الثقة والتقارب في العلاقات السليمة من خلال التواصل والمشاركة وإظهار الأهمية للحب. يمكن استخدام الألعاب لوحية، الكتابات الرومانسية، النشاطات الرياضية، ووسائل التواصل الاجتماعي لزيادة الشعور بالراحة. إذا كانت العلاقة ليست رياضية، يمكنك استخدام الوسائل الأخرى لتشتت الانتباه.'}

In [38]:
import re
from nltk.tokenize import RegexpTokenizer

# Post-process text for ROUGE and BLEU
# def postprocess_text(texts):
#     # Remove punctuation using regex
#     texts = [re.sub(r'[^\w\s]', '', t) for t in texts]
    
#     # # Tokenize into sentences using RegexpTokenizer
#     # sent_tokenizer = RegexpTokenizer(r'\w+')
#     # texts = [" ".join(sent_tokenizer.tokenize(t)) for t in texts]
#     return texts

# Post-process text for ROUGE and BLEU
def postprocess_text(texts):
    text_preds = [(p if p.endswith(("!", "؟", "۔")) else p + "۔") for p in texts]

    sent_tokenizer = RegexpTokenizer(u'[^!؟۔]*[!؟۔]')
    text_preds = ["\n".join([s.strip() for s in sent_tokenizer.tokenize(p)]) for p in text_preds]
    return text_preds

In [39]:
s = summary.split("Summary:")[-1]
print(s)
s = postprocess_text(s)
print(s)


يوجد العديد من الطرق للتعامل مع العلاقة بين الشريكين، بما في ذلك استخدام الثقة والتواصل والمشاركة، كما يمكن استخدام الألعاب للفيديو والكتب والأنشطة الرياضية لزيادة التنوع. يجب أن يكون لديك اتصال مباشر ومباشر مع شريكك حتى تتمكن من التوصل إليه.
['۔', 'ي۔', 'و۔', 'ج۔', 'د۔', '۔', 'ا۔', 'ل۔', 'ع۔', 'د۔', 'ي۔', 'د۔', '۔', 'م۔', 'ن۔', '۔', 'ا۔', 'ل۔', 'ط۔', 'ر۔', 'ق۔', '۔', 'ل۔', 'ل۔', 'ت۔', 'ع۔', 'ا۔', 'م۔', 'ل۔', '۔', 'م۔', 'ع۔', '۔', 'ا۔', 'ل۔', 'ع۔', 'ل۔', 'ا۔', 'ق۔', 'ة۔', '۔', 'ب۔', 'ي۔', 'ن۔', '۔', 'ا۔', 'ل۔', 'ش۔', 'ر۔', 'ي۔', 'ك۔', 'ي۔', 'ن۔', '،۔', '۔', 'ب۔', 'م۔', 'ا۔', '۔', 'ف۔', 'ي۔', '۔', 'ذ۔', 'ل۔', 'ك۔', '۔', 'ا۔', 'س۔', 'ت۔', 'خ۔', 'د۔', 'ا۔', 'م۔', '۔', 'ا۔', 'ل۔', 'ث۔', 'ق۔', 'ة۔', '۔', 'و۔', 'ا۔', 'ل۔', 'ت۔', 'و۔', 'ا۔', 'ص۔', 'ل۔', '۔', 'و۔', 'ا۔', 'ل۔', 'م۔', 'ش۔', 'ا۔', 'ر۔', 'ك۔', 'ة۔', '،۔', '۔', 'ك۔', 'م۔', 'ا۔', '۔', 'ي۔', 'م۔', 'ك۔', 'ن۔', '۔', 'ا۔', 'س۔', 'ت۔', 'خ۔', 'د۔', 'ا۔', 'م۔', '۔', 'ا۔', 'ل۔', 'أ۔', 'ل۔', 'ع۔', 'ا۔', 'ب۔', '۔', 'ل۔', 'ل۔', 'ف۔', 'ي۔', 'د

In [ ]:
split_text = summary.split("### Assistant:", 1)
final_pred = split_text[-1].strip() if len(split_text) > 1 else summary
final_pred = final_pred.replace(tokenizer.eos_token, "")
        
print(f"Test summary: {summary}")

In [17]:
final_pred

'Summarize this arabic text:\n,تشير النصوص إلى أهمية التمارين الرياضية الفترية لتحسين مستوي\n"عبد الحكيم حذاقة-الجزائر عاشت الجزائر مرحلة مفصلية في تاريخها خلال الشهور\nالعشرة الأخيرة من عام 2019، شهدت خلالها ثورة سلمية غير مسبوقة، كانت فيها قوات\nالجيش مرافقة لحراك الشعب، دون أن تراق قطرة دم. وأطاحت الثورة بنظام الرئيس عبد\nالعزيز بوتفليقة بعد عشرين عاما من الحكم، وزج بأقوى رموز عهده من مسؤولين أمنيين\nوحكوميين ورجال مال وراء القضبان، حيث شهدت البلاد محاكمات تاريخية بالجملة في حق\nهؤلاء. وانتهى المخاض العسير طيلة عشرة أشهر بانتخاب رئيس جديد في 12\nديسمبر/كانون الأول، وسط أجواء مشحونة بين المؤيدين والرافضين لشروط الانتخابات،\nحيث لا يزال جزائريون مصرين على الاحتجاج الأسبوعي. وفتحت الوفاة المفاجئة لرئيس\nالأركان الفريق أحمد قايد صالح يوم 23 ديسمبر/كانون الأول باب التكهنات حول ملامح\nالعام الجديد، بالنظر إلى دور الرجل الحاسم في صناعة المرحلة الماضية.\n\n---\nSummary:\nالاستمرار في التمارين الرياضية الفترية يساهم في تعزيز مستويات الصحة العامة، خاصة في حالات القلق من الأمراض المزمنة. \n---

In [13]:
outputs

tensor([[  9190,   5612,    551,    419,  51312,    292,   1467,    510,     11,
         125617,  93543, 124080,  41593, 125155, 128252, 133453,  73441,  94957,
         124916, 123860, 130918,  73441, 124114, 124015,  73441,  56794, 137536,
         126492, 124623,    198,      1, 136961, 123961,  31073, 124176,  68238,
         124169, 124061,     12, 136887,  43982, 126409,  14293, 141912, 125310,
         128707, 127896,  23364, 131645,  73441,  77273, 130570, 124006, 128374,
         124209,  16157,  58656,    198,  31382,  23224, 123941,  25871, 134515,
          63237, 128443,    220,     17,     15,     16,     24,  68785,  52157,
         124837,  14293, 128374, 124006, 125069, 125520,  59842, 124227,  73441,
         128478,  23364, 124125,  12653, 124061,  68785, 128671, 128857,  77703,
         124691,    198,  31382,  33090, 126464, 128707,  95975, 124061,  56794,
          29825, 127471, 129839,  68785, 129193, 125007,  39434, 125806, 136510,
          25871,  44330,  10

In [14]:
summary

'Summarize this arabic text:\n,تشير النصوص إلى أهمية التمارين الرياضية الفترية لتحسين مستوي\n"عبد الحكيم حذاقة-الجزائر عاشت الجزائر مرحلة مفصلية في تاريخها خلال الشهور\nالعشرة الأخيرة من عام 2019، شهدت خلالها ثورة سلمية غير مسبوقة، كانت فيها قوات\nالجيش مرافقة لحراك الشعب، دون أن تراق قطرة دم. وأطاحت الثورة بنظام الرئيس عبد\nالعزيز بوتفليقة بعد عشرين عاما من الحكم، وزج بأقوى رموز عهده من مسؤولين أمنيين\nوحكوميين ورجال مال وراء القضبان، حيث شهدت البلاد محاكمات تاريخية بالجملة في حق\nهؤلاء. وانتهى المخاض العسير طيلة عشرة أشهر بانتخاب رئيس جديد في 12\nديسمبر/كانون الأول، وسط أجواء مشحونة بين المؤيدين والرافضين لشروط الانتخابات،\nحيث لا يزال جزائريون مصرين على الاحتجاج الأسبوعي. وفتحت الوفاة المفاجئة لرئيس\nالأركان الفريق أحمد قايد صالح يوم 23 ديسمبر/كانون الأول باب التكهنات حول ملامح\nالعام الجديد، بالنظر إلى دور الرجل الحاسم في صناعة المرحلة الماضية.\n\n---\nSummary:\nشهدت الجزائر موجة ثورية في الشهور العشرة الأخيرة من عام 2019، مع انتقال السلطة من عبد العزيز بوتفليقة إلى أحمد قايد صالح.

In [4]:
import pandas as pd
df = pd.read_csv('../train/preds.csv')

In [5]:
print(df["Predictions"].iloc[0])


IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIHowWhatWhatWhatIIIWhatWhatWhatWhatWhatWhatWhatWhatWhatWhatWhatHumanHumanWhatWhatWhatWhatHumanWhatHumanHumanHumanHumanHumanHumanHumanHumanWhatHumanHumanHumanHumanHumanHumanHumanmarize this arabic text:
هُ في من الأواقب التي تطلبى حدعاسة ال فيوف م مة من الوع الآخر. العلاقات الجنسرمة.  أناء العلاقاتقة بينواصل بين العلاقات مع خلال تحدث عنماع،تف فيتقترام الآكارك..اق تروج عنعاًمتاع بالبعبةككما..  كنتاولتعك أنظهبار عنك ليس مجردريقة الوحيدة التي يمكنك م محدث،، ف يكون إلى إلى تحدثير في إذا كان هذه ال م التي تريدهم.اً قد المهمتع أن تشعرب في شريكك فيكون ب بزيات م الجنسنسية. هذا هذاعب علىسي تعبية أورلا لائعة لظه النفسريكك ووفيف الضوتره. أنة لمارسة الجنس. يمكنحرص مع معجانب شكما و وماد إلى بعضكسائلائ لسق. بعضصة أو ا أيضا أيضا ت بعضتب التيكوب. إذاساعد هذادركنسسما على نوعجة التاحةتكما إذا إذا كنت ترببي، الجنسج العلاقةور، فح

In [6]:
print(df["Labels"].iloc[0])

Summarize this arabic text يوجد الكثير من الأساليب التي تتعدى ممارسة الجنس لتكوني قريبة من الطرف الآخر في العلاقات السليمة يمكنك بناء الثقة والتقارب في العلاقة من خلال التحدث والاستماع والمشاركة واحترام أفكاركما ومن خلال الخروج معا والاستمتاع بصحبة بعضكما البعض إذا حاول شريكك إخبارك بأن الجنس هو الطريقة الوحيدة التي يمكنكما التقارب بها فقد تحتاجين إلى التفكير فيما إذا كانت هذه العلاقة هي ما تريدينه حقا من الممتع أن تجلسي مع شريكك وتقوما بنشاطات غير جنسية يعد اللعب بألعاب لوحية وسيلة رائعة لإراحة شريكك وتخفيف توتره دون الحاجة لممارسة الجنس اجلسا بجانب بعضكما البعض واستندا على بعض الوسائد وتناوبا على القراءة يمكنكما حتى اختيار الكتب بالتناوب يعتمد قرب جلوسكما على درجة راحتك إذا كنت قلقة بشأن تطور الأمور فحاولي أن تبتعدي قليلا يمكنك الجلوس مثلا على طرف مقابل له مع تلامس قدميكما فقط يمكنكما قراءة بعض القصص الرومانسية لبعضكما من أجل تشتيت الانتباه بعيدا عن الجنس وربما كي تضحكا على مدى رداءة القصص اجلسا معا واستمعا إلى ألبوم ابحثي عن كلمات الأغاني واجلسي بالقرب من شريكك وتمتعي بقرب جسدكما ول